In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
import tensorflow.keras as kr
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder # Transforma uma label em um número inteiro

2024-03-22 09:34:15.478311: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-22 09:34:15.516859: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-22 09:34:15.833885: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-22 09:34:15.836157: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 09:34:16.843030: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

## Read base

In [2]:
base_fraud = pd.read_csv('fraud.csv')

In [3]:
base_fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             555719 non-null  int64  
 1   trans_date_trans_time  555719 non-null  object 
 2   cc_num                 555719 non-null  float64
 3   merchant               555719 non-null  object 
 4   category               555719 non-null  object 
 5   amt                    555719 non-null  float64
 6   first                  555719 non-null  object 
 7   last                   555719 non-null  object 
 8   gender                 555719 non-null  object 
 9   street                 555719 non-null  object 
 10  city                   555719 non-null  object 
 11  state                  555719 non-null  object 
 12  zip                    555719 non-null  int64  
 13  lat                    555719 non-null  float64
 14  long                   555719 non-nu

In [4]:
base_fraud[1:10]

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
1,1,21/06/2020 12:14,3.573030e+15,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",17/01/1990,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,21/06/2020 12:14,3.598220e+15,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",21/10/1970,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,21/06/2020 12:15,3.591920e+15,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,25/07/1987,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,21/06/2020 12:15,3.526830e+15,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,06/07/1955,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0
5,5,21/06/2020 12:15,3.040770e+13,fraud_Daugherty LLC,kids_pets,19.55,Danielle,Evans,F,76752 David Lodge Apt. 064,...,42.1939,-76.7361,520,Psychotherapist,13/10/1991,798db04aaceb4febd084f1a7c404da93,1371816937,41.747157,-77.584197,0
6,6,21/06/2020 12:15,2.131810e+14,fraud_Romaguera Ltd,health_fitness,133.93,Kayla,Sutton,F,010 Weaver Land,...,40.5070,-123.9743,1139,"Therapist, occupational",15/01/1951,17003d7ce534440eadb10c4750e020e5,1371816944,41.499458,-124.888729,0
7,7,21/06/2020 12:15,3.589290e+15,fraud_Reichel LLC,personal_care,10.37,Paula,Estrada,F,350 Stacy Glens,...,43.7557,-97.5936,343,"Development worker, international aid",05/03/1972,8be473af4f05fc6146ea55ace73e7ca2,1371816950,44.495498,-97.728453,0
8,8,21/06/2020 12:16,3.596360e+15,"fraud_Goyette, Howell and Collier",shopping_pos,4.37,David,Everett,M,4138 David Fall,...,41.0001,-78.2357,3688,Advice worker,27/05/1973,71a1da150d1ce510193d7622e08e784e,1371816970,41.546067,-78.120238,0
9,9,21/06/2020 12:16,3.546900e+15,fraud_Kilback Group,food_dining,66.54,Kayla,Obrien,F,7921 Robert Port Suite 343,...,31.6591,-96.8094,263,Barrister,30/05/1956,a7915132c7c4240996ba03a47f81e3bd,1371816971,31.782919,-96.366185,0


, 'category', 'amt', 'street', 'city', 'state', 'city_pop', 'job', 'unix_time', 'lat', 'long', 'merch_lat', 'merch_long', 'is_fraud']

In [5]:
base_fraud = base_fraud[['trans_date_trans_time', 'merchant', 'category', 'amt', 'street', 'city', 'state', 'city_pop', 'job', 'lat', 'long', 'merch_lat', 'merch_long', 'is_fraud']]

In [6]:
base_fraud

,trans_date_trans_time,merchant,category,amt,street,city,state,city_pop,job,lat,long,merch_lat,merch_long,is_fraud
0,21/06/2020 12:14,fraud_Kirlin and Sons,personal_care,2.86,351 Darlene Green,Columbia,SC,333497,Mechanical engineer,33.9659,-80.9355,33.986391,-81.200714,0
1,21/06/2020 12:14,fraud_Sporer-Keebler,personal_care,29.84,3638 Marsh Union,Altonah,UT,302,"Sales professional, IT",40.3207,-110.4360,39.450498,-109.960431,0
2,21/06/2020 12:14,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,9333 Valentine Point,Bellmore,NY,34496,"Librarian, public",40.6729,-73.5365,40.495810,-74.196111,0
3,21/06/2020 12:15,fraud_Haley Group,misc_pos,60.05,32941 Krystal Mill Apt. 552,Titusville,FL,54767,Set designer,28.5697,-80.8191,28.812398,-80.883061,0
4,21/06/2020 12:15,fraud_Johnston-Casper,travel,3.19,5783 Evan Roads Apt. 465,Falmouth,MI,1126,Furniture designer,44.2529,-85.0170,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,31/12/2020 23:59,fraud_Reilly and Sons,health_fitness,43.77,558 Michael Estates,Luray,MO,519,Town planner,40.4931,-91.8912,39.946837,-91.333331,0
555715,31/12/2020 23:59,fraud_Hoppe-Parisian,kids_pets,111.84,572 Davis Mountains,Lake Jackson,TX,28739,Futures trader,29.0393,-95.4401,29.661049,-96.186633,0
555716,31/12/2020 23:59,fraud_Rau-Robel,kids_pets,86.88,144 Evans Islands Apt. 683,Burbank,WA,3684,Musician,46.1966,-118.9017,46.658340,-119.715054,0
555717,31/12/2020 23:59,fraud_Breitenberg LLC,travel,7.99,7020 Doyle Stream Apt. 951,Mesa,ID,129,Cartographer,44.6255,-116.4493,44.470525,-117.080888,0


### Exploração e tratamento dos dados


In [7]:
base_fraud

,trans_date_trans_time,merchant,category,amt,street,city,state,city_pop,job,lat,long,merch_lat,merch_long,is_fraud
0,21/06/2020 12:14,fraud_Kirlin and Sons,personal_care,2.86,351 Darlene Green,Columbia,SC,333497,Mechanical engineer,33.9659,-80.9355,33.986391,-81.200714,0
1,21/06/2020 12:14,fraud_Sporer-Keebler,personal_care,29.84,3638 Marsh Union,Altonah,UT,302,"Sales professional, IT",40.3207,-110.4360,39.450498,-109.960431,0
2,21/06/2020 12:14,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,9333 Valentine Point,Bellmore,NY,34496,"Librarian, public",40.6729,-73.5365,40.495810,-74.196111,0
3,21/06/2020 12:15,fraud_Haley Group,misc_pos,60.05,32941 Krystal Mill Apt. 552,Titusville,FL,54767,Set designer,28.5697,-80.8191,28.812398,-80.883061,0
4,21/06/2020 12:15,fraud_Johnston-Casper,travel,3.19,5783 Evan Roads Apt. 465,Falmouth,MI,1126,Furniture designer,44.2529,-85.0170,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,31/12/2020 23:59,fraud_Reilly and Sons,health_fitness,43.77,558 Michael Estates,Luray,MO,519,Town planner,40.4931,-91.8912,39.946837,-91.333331,0
555715,31/12/2020 23:59,fraud_Hoppe-Parisian,kids_pets,111.84,572 Davis Mountains,Lake Jackson,TX,28739,Futures trader,29.0393,-95.4401,29.661049,-96.186633,0
555716,31/12/2020 23:59,fraud_Rau-Robel,kids_pets,86.88,144 Evans Islands Apt. 683,Burbank,WA,3684,Musician,46.1966,-118.9017,46.658340,-119.715054,0
555717,31/12/2020 23:59,fraud_Breitenberg LLC,travel,7.99,7020 Doyle Stream Apt. 951,Mesa,ID,129,Cartographer,44.6255,-116.4493,44.470525,-117.080888,0


In [8]:
base_fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trans_date_trans_time  555719 non-null  object 
 1   merchant               555719 non-null  object 
 2   category               555719 non-null  object 
 3   amt                    555719 non-null  float64
 4   street                 555719 non-null  object 
 5   city                   555719 non-null  object 
 6   state                  555719 non-null  object 
 7   city_pop               555719 non-null  int64  
 8   job                    555719 non-null  object 
 9   lat                    555719 non-null  float64
 10  long                   555719 non-null  float64
 11  merch_lat              555719 non-null  float64
 12  merch_long             555719 non-null  float64
 13  is_fraud               555719 non-null  int64  
dtypes: float64(5), int64(2), object(7)
m

In [9]:
base_fraud.describe()

,amt,city_pop,lat,long,merch_lat,merch_long,is_fraud
count,555719.000000,5.557190e+05,555719.000000,555719.000000,555719.000000,555719.000000,555719.000000
mean,69.392810,8.822189e+04,38.543253,-90.231325,38.542798,-90.231380,0.003860
std,156.745941,3.003909e+05,5.061336,13.721780,5.095829,13.733071,0.062008
min,1.000000,2.300000e+01,20.027100,-165.672300,19.027422,-166.671575,0.000000
25%,9.630000,7.410000e+02,34.668900,-96.798000,34.755302,-96.905129,0.000000
50%,47.290000,2.408000e+03,39.371600,-87.476900,39.376593,-87.445204,0.000000
75%,83.010000,1.968500e+04,41.894800,-80.175200,41.954163,-80.264637,0.000000
max,22768.110000,2.906700e+06,65.689900,-67.950300,66.679297,-66.952026,1.000000


In [10]:
base_fraud.isna().sum()

trans_date_trans_time    0
merchant                 0
category                 0
amt                      0
street                   0
city                     0
state                    0
city_pop                 0
job                      0
lat                      0
long                     0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [11]:
base_fraud.duplicated().sum()

0

In [8]:
date_time = pd.to_datetime(base_fraud['trans_date_trans_time'], format='%d/%m/%Y %H:%M')
date_time

0        2020-06-21 12:14:00
1        2020-06-21 12:14:00
2        2020-06-21 12:14:00
3        2020-06-21 12:15:00
4        2020-06-21 12:15:00
                 ...        
555714   2020-12-31 23:59:00
555715   2020-12-31 23:59:00
555716   2020-12-31 23:59:00
555717   2020-12-31 23:59:00
555718   2020-12-31 23:59:00
Name: trans_date_trans_time, Length: 555719, dtype: datetime64[ns]

In [9]:
base_fraud['trans_date_trans_time'] = date_time.dt.time
base_fraud

,trans_date_trans_time,merchant,category,amt,street,city,state,city_pop,job,lat,long,merch_lat,merch_long,is_fraud
0,12:14:00,fraud_Kirlin and Sons,personal_care,2.86,351 Darlene Green,Columbia,SC,333497,Mechanical engineer,33.9659,-80.9355,33.986391,-81.200714,0
1,12:14:00,fraud_Sporer-Keebler,personal_care,29.84,3638 Marsh Union,Altonah,UT,302,"Sales professional, IT",40.3207,-110.4360,39.450498,-109.960431,0
2,12:14:00,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,9333 Valentine Point,Bellmore,NY,34496,"Librarian, public",40.6729,-73.5365,40.495810,-74.196111,0
3,12:15:00,fraud_Haley Group,misc_pos,60.05,32941 Krystal Mill Apt. 552,Titusville,FL,54767,Set designer,28.5697,-80.8191,28.812398,-80.883061,0
4,12:15:00,fraud_Johnston-Casper,travel,3.19,5783 Evan Roads Apt. 465,Falmouth,MI,1126,Furniture designer,44.2529,-85.0170,44.959148,-85.884734,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,23:59:00,fraud_Reilly and Sons,health_fitness,43.77,558 Michael Estates,Luray,MO,519,Town planner,40.4931,-91.8912,39.946837,-91.333331,0
555715,23:59:00,fraud_Hoppe-Parisian,kids_pets,111.84,572 Davis Mountains,Lake Jackson,TX,28739,Futures trader,29.0393,-95.4401,29.661049,-96.186633,0
555716,23:59:00,fraud_Rau-Robel,kids_pets,86.88,144 Evans Islands Apt. 683,Burbank,WA,3684,Musician,46.1966,-118.9017,46.658340,-119.715054,0
555717,23:59:00,fraud_Breitenberg LLC,travel,7.99,7020 Doyle Stream Apt. 951,Mesa,ID,129,Cartographer,44.6255,-116.4493,44.470525,-117.080888,0


## Graficos

In [14]:
base_fraud['trans_date_trans_time'].nunique()

1440

### Label encode e one-hot encode

#### Label encode

In [10]:
label_encoder_merchant = LabelEncoder()
label_encoder_street = LabelEncoder()
label_encoder_city = LabelEncoder()
label_encoder_state = LabelEncoder()
label_encoder_job = LabelEncoder()
label_encoder_trans_date_trans_time = LabelEncoder()

In [11]:
base_fraud['merchant'] = label_encoder_merchant.fit_transform(base_fraud['merchant'])
base_fraud['street'] = label_encoder_merchant.fit_transform(base_fraud['street'])
base_fraud['city'] = label_encoder_merchant.fit_transform(base_fraud['city'])
base_fraud['state'] = label_encoder_merchant.fit_transform(base_fraud['state'])
base_fraud['job'] = label_encoder_merchant.fit_transform(base_fraud['job'])
base_fraud['trans_date_trans_time'] = label_encoder_merchant.fit_transform(base_fraud['trans_date_trans_time'])

#### One-hot encode

In [12]:
base_fraud =  pd.get_dummies(base_fraud, columns=['category'])
base_fraud

,trans_date_trans_time,merchant,amt,street,city,state,city_pop,job,lat,long,...,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel
0,734,319,2.86,341,157,39,333497,275,33.9659,-80.9355,...,0,0,0,0,0,0,1,0,0,0
1,734,591,29.84,354,16,43,302,392,40.3207,-110.4360,...,0,0,0,0,0,0,1,0,0,0
2,734,611,41.28,865,61,33,34496,259,40.6729,-73.5365,...,0,1,0,0,0,0,0,0,0,0
3,735,222,60.05,320,764,8,54767,407,28.5697,-80.8191,...,0,0,0,0,0,1,0,0,0,0
4,735,292,3.19,548,247,21,1126,196,44.2529,-85.0170,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555714,1439,507,43.77,531,443,23,519,460,40.4931,-91.8912,...,0,1,0,0,0,0,0,0,0,0
555715,1439,264,111.84,540,401,42,28739,198,29.0393,-95.4401,...,0,0,0,1,0,0,0,0,0,0
555716,1439,496,86.88,126,104,46,3684,294,46.1966,-118.9017,...,0,0,0,1,0,0,0,0,0,0
555717,1439,75,7.99,663,476,12,129,58,44.6255,-116.4493,...,0,0,0,0,0,0,0,0,0,1


## Separando variáveis de decisão e classes

In [13]:
X = base_fraud.drop(columns=['is_fraud'])
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 26 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   trans_date_trans_time    555719 non-null  int64  
 1   merchant                 555719 non-null  int64  
 2   amt                      555719 non-null  float64
 3   street                   555719 non-null  int64  
 4   city                     555719 non-null  int64  
 5   state                    555719 non-null  int64  
 6   city_pop                 555719 non-null  int64  
 7   job                      555719 non-null  int64  
 8   lat                      555719 non-null  float64
 9   long                     555719 non-null  float64
 10  merch_lat                555719 non-null  float64
 11  merch_long               555719 non-null  float64
 12  category_entertainment   555719 non-null  uint8  
 13  category_food_dining     555719 non-null  uint8  
 14  cate

In [14]:
Y = base_fraud['is_fraud'].values
Y

array([0, 0, 0, ..., 0, 0, 0])

In [15]:
X = X.iloc[:, :].values

### One hot encoding

In [16]:
Y = kr.utils.to_categorical(Y,2)
Y

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

### Escalonamento dos valores

In [17]:
scaler_credit = StandardScaler()
X = scaler_credit.fit_transform(X)

In [18]:
X

array([[-0.15658709, -0.11966343, -0.42446311, ..., -0.2851168 ,
        -0.31371195, -0.18004671],
       [-0.15658709,  1.23165562, -0.25233729, ..., -0.2851168 ,
        -0.31371195, -0.18004671],
       [-0.15658709,  1.33101731, -0.17935288, ..., -0.2851168 ,
        -0.31371195, -0.18004671],
       ...,
       [ 1.56708608,  0.75968757,  0.11156401, ..., -0.2851168 ,
        -0.31371195, -0.18004671],
       [ 1.56708608, -1.3318761 , -0.39173496, ..., -0.2851168 ,
        -0.31371195,  5.55411434],
       [ 1.56708608, -1.08347186, -0.19944911, ..., -0.2851168 ,
        -0.31371195, -0.18004671]])

### Divisão das bases em treinamento e teste

In [19]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, Y, test_size = 0.25, random_state = 0)

In [20]:
X_treinamento.shape, y_treinamento.shape

((416789, 26), (416789, 2))

In [21]:
X_teste.shape, y_teste.shape

((138930, 26), (138930, 2))

### Faz o treinamento

In [27]:
mlp_padrao = kr.Sequential([
    Dense(10,activation='relu'),
    Dense(2,activation='softmax')
])

2024-03-22 01:18:27.809103: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-22 01:18:27.819070: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


### Treinando a rede MLP:

In [28]:
mlp_padrao.compile(optimizer='sgd',loss='categorical_crossentropy')

In [29]:
Y

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [30]:
hist = mlp_padrao.fit(X_treinamento,y_treinamento,epochs=500)

Epoch 1/500


In [27]:
presenca = np.all(y_teste == [1, 0], axis=1)

# Conte o número de ocorrências de True
num_ocorrencias = np.count_nonzero(presenca)
num_ocorrencias

138382